# ETL STEAM GAMES

In [159]:
import pandas as pd

## Carga de Data 

In [160]:
df = pd.read_json('Datasets OPS\steam_games.json.gz', compression='gzip', lines=True)
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [161]:
def obtener_informacion_columnas(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].dtype)
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info
informacion_columnas = obtener_informacion_columnas(df)
print(informacion_columnas)

    nombre_campo tipo_datos  no_nulos_%  nulos_%  nulos
0      publisher     object       20.00    80.00  96362
1         genres     object       23.95    76.05  91593
2       app_name     object       26.68    73.32  88312
3          title     object       24.98    75.02  90360
4            url     object       26.68    73.32  88310
5   release_date     object       24.96    75.04  90377
6           tags     object       26.54    73.46  88473
7    reviews_url     object       26.68    73.32  88312
8          specs     object       26.12    73.88  88980
9          price     object       25.54    74.46  89687
10  early_access    float64       26.68    73.32  88310
11            id    float64       26.68    73.32  88312
12     developer     object       23.94    76.06  91609


- En Primera Instancia se puede ver bastantes Filas Completas nulas .

In [164]:
#Remplazo de Filas Nulas  
df.dropna(how='all', inplace=True)

-  Se revisa los tipos de datos de cada columna, a su vez se verifica la existencia de nulos en cada una de ellas.

In [165]:
def obtener_informacion_columnas(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].dtype)
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info
informacion_columnas = obtener_informacion_columnas(df)
print(informacion_columnas)

    nombre_campo tipo_datos  no_nulos_%  nulos_%  nulos
0      publisher     object       74.94    25.06   8052
1         genres     object       89.78    10.22   3283
2       app_name     object       99.99     0.01      2
3          title     object       93.62     6.38   2050
4            url     object      100.00     0.00      0
5   release_date     object       93.57     6.43   2067
6           tags     object       99.49     0.51    163
7    reviews_url     object       99.99     0.01      2
8          specs     object       97.92     2.08    670
9          price     object       95.71     4.29   1377
10  early_access    float64      100.00     0.00      0
11            id    float64       99.99     0.01      2
12     developer     object       89.73    10.27   3299


## Comienzo de transformacion por columnas

#### Transformacion de columna id

In [166]:
print(df['id'].value_counts()) #Vemos si hay un valor duplicado
print('-'*50)
print((df['id'].value_counts() > 1).value_counts()) #Cuantos valores duplicados hay

id
612880.0    2
670290.0    1
767400.0    1
20.0        1
772590.0    1
           ..
773650.0    1
774271.0    1
773690.0    1
774279.0    1
643980.0    1
Name: count, Length: 32132, dtype: int64
--------------------------------------------------
count
False    32131
True         1
Name: count, dtype: int64


In [168]:
#Vemos si hay id nulos
df[df['id'].isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


- Se encontraron dos filas con id nulo, una solo posee valores de url y precio, ademas el url es de la store de steam por lo tanto se eliminara.
- La otra fila con id nulo estaba replicada de otra fila, ya que tiene los mismos valores, por esto tambien se eliminara

In [169]:
#Eliminacion de  valores duplicados y nulos
df.drop_duplicates(subset='id', inplace=True) 
df.dropna(subset='id', inplace=True)

#Cambio el tipo de dato
df['id'] = df['id'].astype(int) 

#### Transformacion de columna genres


In [170]:
#Vemos los valores unicos que tenemos
df['genres'].value_counts()

genres
[Action]                                                                  1879
[Action, Indie]                                                           1650
[Simulation]                                                              1396
[Casual, Simulation]                                                      1359
[Action, Adventure, Indie]                                                1082
                                                                          ... 
[Sports, Simulation]                                                         1
[Action, Strategy, Simulation]                                               1
[Free to Play, Action, Strategy, Indie, Sports]                              1
[Action, Adventure, RPG, Indie]                                              1
[Action, Casual, Free to Play, Massively Multiplayer, RPG, Simulation]       1
Name: count, Length: 883, dtype: int64

- Tags y Genre tienen informacion muy similar que se tienen datos que pueden rellenar los nulos que contiene genre

In [171]:
#Interpeto que tiene una relacion tags y geners por el contenido que tienen
df['genres'] = df['genres'].fillna(df['tags'])
# borro el resto de los nulos
df.dropna(subset='genres', inplace=True) 

#### Transformacion de columna price

In [173]:
#Vemos los valores unicos que tenemos
df['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, None, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 109.99, 149.99,
       771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 119.99, 3.0, 15.0, 5.49, 23.99,
       49.0, 20.99, 10.93, 1.3900000000000001,
       'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0, 9.0, 234.99,
       1.9500000000000002, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49,

In [174]:
#Estos casos partiuclares los ponemos en numero
df.loc[df['price'] == 'Starting at $499.00', 'price'] = 499.00
df.loc[df['price'] == 'Starting at $449.00', 'price'] = 449.00

- Se Reemplaza la columna original con la nueva versión convertida a números y con 0.0 en todos los casos donde no se pudo convertir.

In [175]:
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0.0)

#### Transformacion de columna relase data

In [177]:
#Vemos los valores unicos que tenemos
df['release_date'].unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2016-11-19',
       'January 2018', '2018-10-01'], dtype=object)

In [180]:
# Filtrar valores que NO tengan el patrón de fecha con guiones
valores_raros = df.loc[~df['release_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False), 'release_date'].unique()

print(len(valores_raros))
valores_raros


82


array(['2018', '2017', 'None', 'Soon..', '1997', '1998', '2016', '2006',
       'Beta测试已开启', '2005', '2003', '2007', '2002', '2000', '1995',
       '1996', '1994', '2001', '1993', '2004', '1999', '2008', '2009',
       '1992', '1989', '2010', '2011', '2013', '2012', '2014', '1983',
       '1984', '2015', '1990', '1988', '1991', '1985', '1982', '1987',
       'Coming Soon', 'TBA', "When it's done", 'coming soon', '1981',
       'soon', '14 July', '1986', 'To be Announced', 'TBD', 'Soon',
       'Please wait warmly', 'early access', 'SOON',
       'Coming Soon/Próximamente', '2021', '5275', 'Coming soon',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       '"""Soon"""', 'Demo coming soon.', 'Not yet available',
       "C'est bientôt...                    (ou pas)", '2019',
       'Datachunks conflicted. Be vigiliant.', 'Demo is available now!',
       '1975', '1970', 'Play Beta in demo!', 'Coming Soon!',
       'To be announced', 'W

In [179]:
import re

def extraer_anio(fecha):
    match = re.search(r'\d{4}', fecha)
    return match.group(0) if match else fecha

df['release_date'] = df['release_date'].astype(str).apply(extraer_anio)

In [181]:
df['release_date'] = pd.to_numeric(df['release_date'], errors='coerce')

- Borramos los nulos y verificamos que haya quedado bien el formato

In [185]:
#borramos el resto de los nulos
df.dropna(subset='release_date', inplace=True) 
df['release_date'] = df['release_date'].astype(int) 

#### Tranformacion de columna developer

In [191]:
# Vemos los valores unicos
df['developer'].unique()

array(['Kotoshiro', 'Secret Level SRL', 'Poolians.com', ...,
       'Oscar Ortigueira López,OrtiGames/OrtiSoft', 'INGAME',
       'Bidoniera Games'], dtype=object)

In [193]:
# Vemos los nulos
df[df['developer'].isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88336,None,"[Design & Illustration, Tutorial]",Robotpencil Presents: Understanding 3D for Con...,Robotpencil Presents: Understanding 3D for Con...,http://store.steampowered.com/app/777910/Robot...,2018,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777910/reviews/?...,None,4.99,0.0,777910,None
88427,Retroism,[Simulation],Tycoon City: New York,Tycoon City: New York,http://store.steampowered.com/app/9730/Tycoon_...,2006,"[Simulation, Management, City Builder]",http://steamcommunity.com/app/9730/reviews/?br...,[Single-player],9.99,0.0,9730,None
88576,"ValuSoft, Retroism","[Simulation, Sports]",Hunting Unlimited 2010,Hunting Unlimited 2010,http://store.steampowered.com/app/12690/Huntin...,2009,"[Simulation, Hunting, Sports, America]",http://steamcommunity.com/app/12690/reviews/?b...,[Single-player],9.99,0.0,12690,None
88614,Meridian4,"[Action, Racing]",Crash Time 2,Crash Time 2,http://store.steampowered.com/app/11390/Crash_...,2009,"[Racing, Action, Open World, Singleplayer, Dri...",http://steamcommunity.com/app/11390/reviews/?b...,[Single-player],9.99,0.0,11390,None
88617,"ValuSoft, Retroism",[Simulation],18 Wheels of Steel: Extreme Trucker,18 Wheels of Steel: Extreme Trucker,http://store.steampowered.com/app/33730/18_Whe...,2009,"[Simulation, Driving]",http://steamcommunity.com/app/33730/reviews/?b...,[Single-player],9.99,0.0,33730,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120297,None,"[Design & Illustration, Tutorial]",Robotpencil Presents: Start with Color,Robotpencil Presents: Start with Color,http://store.steampowered.com/app/775660/Robot...,2017,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/775660/reviews/?...,None,3.99,0.0,775660,None
120428,None,"[Design & Illustration, Tutorial]",Robotpencil Presents: Exercise: Brushwork,Robotpencil Presents: Exercise: Brushwork,http://store.steampowered.com/app/775640/Robot...,2018,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/775640/reviews/?...,None,3.99,0.0,775640,None
120429,None,"[Design & Illustration, Tutorial]",Robotpencil Presents: Creative Composition,Robotpencil Presents: Creative Composition,http://store.steampowered.com/app/777930/Robot...,2018,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777930/reviews/?...,None,3.99,0.0,777930,None
120430,None,[Movie],The Gamble House,The Gamble House,http://store.steampowered.com/app/775370/The_G...,2016,[Movie],http://steamcommunity.com/app/775370/reviews/?...,[Captions available],4.99,0.0,775370,None


- publisher tiene algunas similitudes con developer, los nulos los relleno con esta informacion con los que se pueden

In [194]:
# Aplico a los none como los developer aquel que lo publico (publisher)
df.loc[(df['developer'].isna()) & (df['publisher'].notnull()),'developer'] = df[(df['developer'].isna()) & (df['publisher'].notnull())]['publisher']

In [195]:
# Borro el resto de los null
df.dropna(subset='developer', inplace=True)

# pongo la primera letra de todas las palabras en mayuscula
df['developer']= df['developer'].apply(lambda x: x.rstrip().title())

#### Retoques finales

- Verificamos los nulos que quedaron

In [196]:
df.isna().sum()

publisher       4776
genres             0
app_name           0
title              0
url                0
release_date       0
tags              24
reviews_url        0
specs            286
price              0
early_access       0
id                 0
developer          0
dtype: int64

- Borro las columnas que me resultan que no tienen informacion imporante

In [197]:
# Borro columnas que no tienen informacion importante
df.drop(columns=['publisher','url','app_name','tags','reviews_url','specs', 'early_access'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,genres,title,release_date,price,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,0.00,643980,Secret Level Srl
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,0.00,670290,Poolians.Com
3,"[Action, Adventure, Casual]",弹炸人2222,2017,0.99,767400,彼岸领域
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018,3.99,772540,Trickjump Games Ltd


In [200]:
def obtener_informacion_columnas(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].dtype)
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info
informacion_columnas = obtener_informacion_columnas(df)
print(informacion_columnas)

   nombre_campo tipo_datos  no_nulos_%  nulos_%  nulos
0        genres     object       100.0      0.0      0
1         title     object       100.0      0.0      0
2  release_date      int64       100.0      0.0      0
3         price    float64       100.0      0.0      0
4            id      int64       100.0      0.0      0
5     developer     object       100.0      0.0      0


### Exportamos el dataset 


In [206]:
df.to_json('../Datasets/Steam_Games_Limpio.json.gz', compression='gzip')